In [ ]:
import sunpy.map

In [ ]:
from itipy.data.dataset import BaseDataset
from itipy.data.editor import LoadMapEditor, NormalizeRadiusEditor, RemoveOffLimbEditor, MapToDataEditor, NanEditor, \
    NormalizeEditor, ReshapeEditor, AIAPrepEditor

class AIADataset(BaseDataset):
    """
    Dataset for SDO/AIA data

    Args:
        data: Data
        wavelength (int): Wavelength
        resolution (int): Resolution
        ext (str): File extension
        calibration (str): Calibration type
        **kwargs: Additional arguments
    """

    def __init__(self, data, wavelength, resolution=2048, ext='.fits', calibration='auto', **kwargs):

        editors = [LoadMapEditor(),
                   NormalizeRadiusEditor(resolution),
                   AIAPrepEditor(calibration=calibration),
                   MapToDataEditor(),
                   ReshapeEditor((1, resolution, resolution))]
        super().__init__(data, editors=editors, ext=ext, **kwargs)

In [ ]:
import os
import glob
from sunpy.map import Map
from astropy.io import fits
from astropy.io.fits import Header, PrimaryHDU
import numpy as np
import tqdm as tqdm
import csv

input_folder = '/mnt/data/SDO-AIA/131'
output_folder = '/home/griffingoodwin/2025-HL-Flaring-MEGS-AI'
os.makedirs(output_folder, exist_ok=True)

# 🔎 Collect all .fits files
fits_files = glob.glob(os.path.join(input_folder, '*.fits'))
fits_files = fits_files[:10]



editors = [
    LoadMapEditor(),
    NormalizeRadiusEditor(512),
    AIAPrepEditor(calibration='auto'),
    MapToDataEditor()
]

dataset = BaseDataset(fits_files, editors=editors, ext='.fits', wavelength=131, resolution=512)

In [ ]:
for fits_file in tqdm.tqdm(fits_files, desc="Processing FITS files"):

    data, meta = dataset.convertData([fits_file])
    meta_header = meta['header']
    del meta_header['keycomments']
    print(meta_header)
    # Save the processed data as a FITS file
    output_file = os.path.join(output_folder, os.path.basename(fits_file))
    fits.writeto(output_file, data, header=Header(meta_header), overwrite=True)


In [ ]:
sunpy.map.Map('/home/griffingoodwin/2025-HL-Flaring-MEGS-AI/2023-07-12T05:15:00.fits').peek()